This is how to scrap imdb top 10000 films and get high quality posters



In [ ]:
import json
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse

def main(start):
    url = f"https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&count=100&start={start}&ref_=adv_nxt"
    response = requests.get(url)
    soup = BeautifulSoup(response.content , "html.parser")

    movies = []

    movies_data = soup.findAll("div" , attrs={"class":"lister-item mode-advanced"})

    for movie in movies_data:
        name = movie.h3.a.text
        release_year = movie.h3.find("span", class_ = "lister-item-year text-muted unbold").text.replace("(", "").replace(")", "")
        movie_time = movie.p.find("span", class_ = "runtime").text.replace(' min', '')
        movie_rating = movie.find("div", class_ = "inline-block ratings-imdb-rating").text.replace("\n", "")
        genre = movie.p.find("span", class_ = "genre").text.strip().replace("\n" , "")
        description = movie.find_all("p", {"class":  "text-muted"})[1].text.strip("\n")
        crew = movie.find("p", {"class":  ""}).find_all("a")
        director = crew[0].text
        stars = crew[1 : len(crew)]
        stars = [star.text for star in stars]
        poster = movie.find("div", {"class": "lister-item-image float-left"}).find("img", "loadlate").get('loadlate')
    
        movies.append({"name" : name , "year" : release_year ,
        "runtime" : movie_time ,"rating" : movie_rating , "genre" : genre , "image": poster, 
        "description" : description, "director" : director, "stars" : stars})

    
    return json.dumps(movies, indent= 4)
    

if __name__ == "__main__":
    films = ""
    for i in range(1, 10001,100):
        
        films += main(i)
        print(f"done {i}")
    
    with open("films.json", "w") as f:
        f.write(films)

First we define a main function that takes a parameter (start) it takes this parameter and requests a hundred film starting from that parameter start
first we request the url then we parse the html content with beautiful soup library
We then search the content of the page for all divs that has a class of lister-item mode-advanced we only know that it has the data from looking through the page
We then iterate over each div in these data
Inside each div we will be searching for name, year of release, the run time, the rating, the genre, director, description of the film, the stars and the poster
We then transform this to json data and return them 
After that in our main code we will be iterating from 1 to 10001 and pass that as the main functuion as the start variable
after that we write all that json in a file called films.json.

We could be enough with this however the quality of the poster images are really bad we will be continuing with some more logic to get high quality posters.
To get the hd posters we will have to go to each film page specifically and parse that image.
First to visit each page dynamcilly we need the specific id for each film that imdb puts on them this can be done easily as it can be found with the data that we parsed before so we will implement the same logic only looking in other place.
The code below will get us the 10000 ids and store it in a csv file so we can use it in the next logic

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse

def main(start):
    url = f"https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&count=100&start={start}&ref_=adv_nxt"
    response = requests.get(url)
    soup = BeautifulSoup(response.content , "html.parser")

    movies = []

    movies_data = soup.findAll("div" , attrs={"class":"lister-item mode-advanced"})

    for movie in movies_data:
        imdb_id = movie.find("div", class_ = "lister-item-image float-left").a["href"]
        imdb_id = imdb_id.replace("/", "").replace("title", "")
        movies.append(imdb_id)

    return movies  

if __name__ == "__main__":
    films = []
    for i in range(1, 10001,100):
        films += main(i)
        print(f"done {i}")
    
    with open("links.csv", "w") as f:
        for film in films:
            f.write(f'{film}\n')

First we write the scrape_link function that takes a url as parameter and requests the page we parse finding the image link and return it.
We then write the bigger logic we open the links csv file and then open the films csv file to replace the poster links we read the films and then iterate over them one by one.


In [ ]:
import json
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
import urllib.parse

def scrape_link(url):

    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

    soup = BeautifulSoup(response.content , "html.parser")

    a = soup.find('a', class_="ipc-lockup-overlay ipc-focusable")

    return a["href"]


counter = 1


with open("links.csv", "a") as file:

    with open('films.csv', "r") as f:
        films = f.readlines()

    for film in films:
        film = film.replace("\n", "")

        link = scrape_link(f"https://www.imdb.com/title/{film}")

        response_2 = requests.get(f"https://www.imdb.com{link}", headers={'User-Agent': 'Mozilla/5.0'})
        

        soup = BeautifulSoup(response_2.content , "html.parser")

        div = soup.find('div', class_ = "sc-7c0a9e7c-2 kEDMKk")
        if div:
            file.write(f"{div.img['src']}\n")
            print(f"done{counter}")

        else:
            file.write("\n")
            print(f"done{counter} blank")

        counter += 1
    






